In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

                                                                            CUSTOMER SEGMENTATİON
# Before we begin;
  * If you like my work, please upvote this kernel because it will motivated me for more research and work. 
  
# Introduction;
  * Firstly we will explore the dataset and we will make some visualizations.
  * We will create a new dataset which is about rfm analysis. With the dataset we are going to segment the customers with some properties.
  * We will use the Kmeans algorithm and several dimensionality reduction techniques.

In [ ]:
data = pd.read_csv('../input/sample-sales-data/sales_data_sample.csv', encoding = 'unicode_escape')
print("data shape is:", data.shape, "*"*50)
data.head(3)

In [ ]:
describe = data.describe().T
describe.style.background_gradient(cmap = "Oranges")

In [ ]:
columns  = data.columns
nunique = pd.DataFrame(data[columns].nunique())
nunique.style.background_gradient(cmap="turbo")

# FOR RFM ANALYSIS
1. "SALES" column is for "monetory"
2. "ORDERDATE" column is for "recency"
3. "QUANTITYORDERED" column is for "frequency"

In [ ]:
print("Total Missing Values Numbers of columns;")
isnull = pd.DataFrame(data.isnull().sum())
isnull.style.background_gradient(cmap="ocean")

In [ ]:
fig = px.pie(data, names='COUNTRY', title='Country Pie Chart')
fig.show()

In [ ]:
# boş değere sahip olan 3 kolon için herhangi bir işlem yapmayacağım. Oranları çok fazla ve bu projenin ana odağı değil. 
# Ancak ilerideki PCA tekniği için missing value'larda kurtulmam gerek

addresline2_missing_value_rates = data["ADDRESSLINE2"].isnull().sum() * 100 / len(data)
state_missing_value_rates = data["STATE"].isnull().sum() * 100 / len(data)
terrıtory_missing_value_rates = data["TERRITORY"].isnull().sum() * 100 / len(data)

print("ADDRESSLINE2 missing value rates % : ",round(addresline2_missing_value_rates,3))
print("STATE missing value rates % : ",round(state_missing_value_rates,3))
print("TERRITORY missing value rates % : ",round(terrıtory_missing_value_rates,3))

data = data.drop(["ADDRESSLINE2","STATE","TERRITORY"], axis=1)


# Burada bir kaç kolonumuzun eksik veriye sahip olduğunu görebiliriz. Ancak bu kolonlar RFM analizi veya kümeleme için önemli olan kolonlardan değil. Bu yüzden bu adımları atlayıp konumuza geri dönelim.

# recency yani zamanlama, yenilik kavramını sağlamak için tarih ile ilgili dataları düzenleyelim.

In [ ]:
data["ORDERDATE"] = pd.to_datetime(data["ORDERDATE"]) 

print("Last date in data:", data["ORDERDATE"].max())
print("First date in data:", data["ORDERDATE"].min())
#son tarihi referans alarak ilgili işlemlerimi yapıyorum
import datetime as dt
Now = dt.datetime(2005,5,31)

In [ ]:
pd.to_datetime(data["ORDERDATE"]) 

In [ ]:
columns  = data.columns
nunique = pd.DataFrame(data[columns].nunique())
nunique.style.background_gradient(cmap="turbo")

In [ ]:
data["recency_day"] = (Now - data["ORDERDATE"]).astype(str)
data[["recency_day", "B"]] = data["recency_day"].str.split(" ",expand = True)
data = data.drop(["B", "ORDERDATE"], axis=1)

# Tarih değişkeninin gün bazında değişimini merak ediyorum ve bunun için bir plot çizdirelim.

In [ ]:
plt.figure(figsize=(30,10))
sns.set_style("darkgrid")
sns.histplot(data['recency_day'], bins=50,
            discrete = True,
            color = "#21DA4F")
plt.xticks(rotation=45)
plt.show

In [ ]:
data.head()

# Price analizi yapmadan önce gerekli temizlemeleri ve kontrolleri yapalım.
Bu kontroller;

* 1-) Negatif price değeri var mı?
* 2-) Nan value'lar mevcutsa bunların doğru bir şekilde doldurulması.
* 3-) İlgili Fiyat değerlerinin dağılımlarına bakalım. Normal dağılım bizim istediğimiz bir durumdur.
Price değerleri için normallik ve istatistiksel testlerine bakalım.



In [ ]:
import numpy as np
from scipy.stats import kurtosis
from scipy.stats import skew
print( 'excess kurtosis of normal distribution (should be 0): {}'.format( kurtosis(data["SALES"]) ))
print( 'skewness of normal distribution (should be 0): {}'.format( skew(data["SALES"]) ))

# The values for asymmetry and kurtosis between -2 and +2 are considered acceptable in order to prove normal univariate distribution (George & Mallery, 2010). 

In [ ]:
temp  = pd.DataFrame(data["SALES"]).describe().T
temp.style.background_gradient(cmap="autumn_r")
#Herhangi bir negatif değerimiz yok. Bu kontrol analizimiz için oldukça önemli.

In [ ]:
data['price_log'] = np.log(data['SALES']+1) 
#Bunun için daha farklı dönüşüm tipleri de bulunmaktadır.
fig, axarr = plt.subplots(2, 1, figsize=(24, 12))

sns.histplot(data["SALES"], bins=100,kde=True,ax=axarr[0])
axarr[0].set_title("Sales column graph", color="#f60000", size=20)
axarr[0].set_ylabel('Count', fontsize = 20, color="#2d242e")
axarr[0].set_xlabel('Sale', fontsize = 20, color="#2d242e")

sns.histplot(data['price_log'], bins=80, kde=True, ax=axarr[1])
axarr[1].set_title("Sales column after log transformation", color="#f60000", size=20)
axarr[1].set_ylabel('Count', fontsize = 20, color="#2d242e")
axarr[1].set_xlabel('Log_sale', fontsize = 20, color="#2d242e")
fig.show()

In [ ]:
print( 'excess kurtosis of normal distribution (should be 0): {}'.format( kurtosis(data["price_log"]) ))
print( 'skewness of normal distribution (should be 0): {}'.format( skew(data["price_log"]) ))

# Sanki Daha Normal !

# Sıradaki adım müşteri bazında toplam harcamaları, sıklık ve yenilik bazında gruplayalım.
* Bir müşterinin birden çok alışverişi veya tarih değişkeni olabilir. Bu değerlerin daha sonra ortalamasını alıp nihai hedefimize ulaşabiliriz.

In [ ]:
data_rfm = data.groupby(["CUSTOMERNAME"]).agg({
    "ORDERNUMBER" :"count",
    "SALES":  "sum",
    "recency_day":"min"})  # burada min değerini seçerek müşterinin en son ne zaman işlem yaptığı bilgisine ulaşıyorum.

data_rfm.rename(columns={'recency_day': 'Recency',
                   'ORDERNUMBER': 'Frequency',
                   'SALES': 'Monetary'}, inplace=True)

In [ ]:
data_rfm.iloc[:5].style.background_gradient(cmap="Pastel1", low=0.50, high=1.0)

# Lets make a RFM_score variable 

In [ ]:
data_rfm["Recency"] = data_rfm["Recency"] .astype(int)

In [ ]:
zamansallık = pd.qcut(data_rfm["Recency"], q = 5, labels = range(1,6)).astype(str)
sıklık = pd.qcut(data_rfm["Frequency"], q = 5,    labels = range(1,6)).astype(str)
parasallık = pd.qcut(data_rfm["Monetary"], q = 5, labels = range(1,6)).astype(str)

data_rfm["R"] = zamansallık
data_rfm["F"] = sıklık
data_rfm["M"] = parasallık
data_rfm["RFM_score"] = data_rfm["R"]+ data_rfm["F"] +data_rfm["M"]
data_rfm.iloc[:5].style.background_gradient(cmap="Pastel1", low=0.50, high=1.0)

# puanları genel olarak toplayıp ona göre aksiyon almak bizi yanıltabilir. O yüzden R, F ve m başlıklarını ayrı ayrı değerlendirmemiz gerektiğini düşünüyorum
# RFM score'ları 511 ve 133 olan müşterilerin genel skorları aynıdır yani 7. Ancak bu iki müşteri grubu için aynı kampanyaları yapmak pek de verimli olmayabilir.
# Strateji olarak;
* örnek olarak rfm skorları şu şekilde olan müşteriler için ;
* RFM_score = xx5  => Bu segmentasyondaki müşteriler için en pahalı ürünleri pazarlamaya çalış
* RFM_score = 311 / 411 segmentinde olan müşteriler için agresif indirim kampanyaları düzenlenerek, ilgili müşteriler tekrar şirkete kazandırılabilir.

In [ ]:
data_rfm[["Frequency","Monetary", "Recency"]] = data_rfm[["Frequency","Monetary", "Recency"]].astype(float)

In [ ]:
R = pd.DataFrame(data_rfm["R"].value_counts())
M = pd.DataFrame(data_rfm["M"].value_counts())
F = pd.DataFrame(data_rfm["F"].value_counts())

In [ ]:

fig = make_subplots(rows=1, cols=3, specs=[[{"type": "pie"}, {"type": "pie"},{"type": "pie"}]])


fig.add_trace(go.Pie(
     values=[19,18,19,17,19],
     labels=["1","2","3","4","5"],
     domain=dict(x=[0, 0.5]),
     name="GHG Emissions",
     title="Müşteri segmentasyonundaki R(recency) dağılımı"), 
     row=1, col=1)

fig.add_trace(go.Pie(
     values=[19,18,18,18,19],
     labels=["1","2","3","4","5"],
     domain=dict(x=[0, 0.5]),
     name="GHG Emissions",
     title="Müşteri segmentasyonundaki M(mondetory) dağılımı"), 
     row=1, col=2)

fig.add_trace(go.Pie(
     values=[20,18,19,17,18],
     labels=["1","2","3","4","5"],
     domain=dict(x=[0, 0.5]),
     name="GHG Emissions",
     title="Müşteri segmentasyonundaki F(frequency) dağılımı"), 
     row=1, col=3)

fig.show()

# Müşterilerin 3 kriter içinde yakın dağılım gösterdiğini söyleyebiliriz.

# Kmeans algoritmasının çok daha verimli çalıştırmak için bazı düzenlemeler yapmalıyız. Bunun için;

1. İlgili değişkenlerin normal dağılıma sahip olması,
2. Değişkenlerin aynı veya yakın ortalamada olması,
3. Değişkenlerin aynı veya yakın varyans değerlerine sahip olması.

In [ ]:
plt.style.use('ggplot')
fig, axarr = plt.subplots(1, 3, figsize=(32, 10))

axarr[0].hist(data_rfm['Recency'], bins=20, histtype= 'bar')
axarr[0].set_title("Recency")

axarr[1].hist(data_rfm['Frequency'], bins=20, histtype= 'stepfilled', color="#31CC57")
axarr[1].set_title("Frequency")

axarr[2].hist(data_rfm['Monetary'], bins=20,histtype= 'step', linewidth=3.2, color="#DE50DE")
axarr[2].set_title("Monetary")
plt.show()


1. Satış fiyatlarını daha önce logaritmik dönüşüm ile normal dağılıma yaklaştırmıştık.
2. Aynı adımları diğer değişkenler için yapalım.

In [ ]:
data_rfm["Recency"] = np.log(data_rfm["Recency"]+1)
data_rfm["Frequency"] = np.log(data_rfm["Frequency"]+1)
data_rfm["Monetary"] = np.log(data_rfm["Monetary"]+1)

In [ ]:
plt.style.use('ggplot')
fig, axarr = plt.subplots(1, 3, figsize=(32, 10))

axarr[0].hist(data_rfm['Recency'], bins=20, histtype= 'bar')
axarr[0].set_title("Recency")

axarr[1].hist(data_rfm['Frequency'], bins=20, histtype= 'stepfilled', color="#31CC57")
axarr[1].set_title("Frequency")

axarr[2].hist(data_rfm["Monetary"], bins=20,histtype= 'step', linewidth=3.2, color="#DE50DE")
axarr[2].set_title("Monetary")
plt.show()


In [ ]:
data_rfm.iloc[:5].style.background_gradient(cmap="autumn_r", low=0.50, high=1.0)

# Why do we need scaling?
Machine learning algorithm just sees number  if there is a vast difference in the range say few ranging in thousands and few ranging in the tens, and it makes the underlying assumption that higher ranging numbers have superiority of some sort. So these more significant number starts playing a more decisive role while training the model.

In [ ]:
sc = StandardScaler()
scale_data =sc.fit_transform(data_rfm.iloc[:,:3])
scale_data = pd.DataFrame(scale_data, index = data_rfm.index, columns = ["Frequency", "Monetary", "Recency"])

In [ ]:
temp  = pd.DataFrame(scale_data).describe().T
temp.style.background_gradient(cmap="autumn_r")

1. Değişken dağılımları normal dağılıma yaklaştırıldı
2. StandardScale ile ise ortalamaları 0 yaparak Kmeans algoritması iiçn uygun çalışma ortamı hazırladık.

# Görselleştirme için hazır bir kütüphane kullandım. Buradaki siyah çizgi en uygun number_of_cluster' a denk gelmektedir. 
* Bu nokta aynı zamanda grafikteki en yüksek açıya sahip noktadır. (Dirsek Kuralı)

In [ ]:
print('Elbow Method to determine the number of clusters to be formed:')
Elbow_M = KElbowVisualizer(KMeans(), k=15)
plt.figure(figsize=(22,10))
Elbow_M.fit(scale_data)
Elbow_M.show()

In [ ]:
# En uygun sınıf sayısına göre kümelememizi yapalım

kmeans = KMeans(n_clusters = 5, random_state = 1)
kmeans.fit(scale_data)
cluster_labels = kmeans.labels_
data_rfm = data_rfm.assign(Cluster=cluster_labels)
data_rfm.head()

In [ ]:
data_rfm["Recency"] = np.exp(data_rfm["Recency"])
data_rfm["Frequency"] = np.exp(data_rfm["Frequency"])
data_rfm["Monetary"] =np.exp(data_rfm["Monetary"])

In [ ]:
data_rfm[["Frequency","Monetary", "Recency"]] = data_rfm[["Frequency","Monetary", "Recency"]].astype(float)
grouped = data_rfm.groupby(['Cluster'])

# Calculate average RFM values and segment sizes per cluster value
grouped.agg({
    'Recency': ['mean','std'],
    'Frequency': ['mean','std'],
    'Monetary': ['mean', 'count']
  }).round(1)

In [ ]:
data_dim = data.merge(data_rfm[['Cluster']], on="CUSTOMERNAME", how="left")

# We will make it numeric for dimension reduction.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label_data = ["PRODUCTCODE","CUSTOMERNAME","PHONE","ADDRESSLINE1","CITY",
              "POSTALCODE","CONTACTLASTNAME","COUNTRY","CONTACTLASTNAME",
              "CONTACTFIRSTNAME","DEALSIZE","recency_day","STATUS","PRODUCTLINE"]
for i in label_data:
    data[i] =  data[i].astype(str)
    data[i] = le.fit_transform(data[i].values)
    
data.head()

In [ ]:
#PCA 
pca = PCA()
pipe = Pipeline([('scaler', StandardScaler()), ('pca', pca)])
X_pca = pipe.fit_transform(data)

#t-sne
tsne = TSNE()
pipe = Pipeline([("scaler", StandardScaler()), ('tsne', tsne)])
X_tsne = pipe.fit_transform(data)

In [ ]:
plt.style.use('ggplot')
fig, axarr = plt.subplots(1, 2, figsize=(32, 10))

fig.suptitle("Dimensionality Reduction techniques", color ="red", size = 28)
axarr[0].scatter(X_pca[:,0], X_pca[:,1],alpha=0.8,edgecolors="#F50D2F" ,linewidths=0.8, c="#49BAF1")
axarr[0].set_title("Ham data işlenmeden PCA uygulanması" ,color="red")
axarr[0].set_ylabel("2.düzlem", color ="black")
axarr[0].set_xlabel("1.düzlem", color ="black")
axarr[0].set_yticklabels([])
axarr[0].set_xticklabels([])
axarr[0].grid(False)

axarr[1].scatter(X_tsne[:,0], X_tsne[:,1],alpha=0.8,edgecolors="#F50D2F" ,linewidths=0.8, c="#49BAF1")
axarr[1].set_title("Ham data işlenmeden t-SNE uygulanması" ,color="red")
axarr[1].set_ylabel("2.düzlem", color ="black")
axarr[1].set_xlabel("1.düzlem", color ="black")
axarr[1].set_yticklabels([])
axarr[1].set_xticklabels([])
axarr[1].grid(False)
fig.show()


# son olarak ham data ile cluster kolonunu birleştirerek bir görselleştirme yapalım.

In [ ]:

label_data = ["PRODUCTCODE","CUSTOMERNAME","PHONE","ADDRESSLINE1","CITY",
              "POSTALCODE","CONTACTLASTNAME","COUNTRY","CONTACTLASTNAME",
              "CONTACTFIRSTNAME","DEALSIZE","recency_day","STATUS","PRODUCTLINE"]
for i in label_data:
    data_dim[i] =  data_dim[i].astype(str)
    data_dim[i] = le.fit_transform(data[i].values)
    
y = data_dim["Cluster"]

#PCA 
pipe = Pipeline([('scaler', StandardScaler()), ('pca', pca)])
X_pca = pipe.fit_transform(data_dim.iloc[:,:23])

pipe = Pipeline([("scaler", StandardScaler()), ('tsne', tsne)])
X_tsne = pipe.fit_transform(data_dim.iloc[:,:23])

print(X_tsne.shape)

In [ ]:
plt.style.use('ggplot')
fig, axarr = plt.subplots(1, 2, figsize=(32, 10))

fig.suptitle("Dimensionality Reduction techniques", color ="red", size = 28)
axarr[0].scatter(X_pca[:,0], X_pca[:,1],alpha=0.8,edgecolors="#F50D2F" ,linewidths=0.8, c=y)
axarr[0].set_title("Ham data işlenmeden PCA uygulanması" ,color="red")
axarr[0].set_ylabel("2.düzlem", color ="black")
axarr[0].set_xlabel("1.düzlem", color ="black")
axarr[0].set_yticklabels([])
axarr[0].set_xticklabels([])
axarr[0].grid(False)

axarr[1].scatter(X_tsne[:,0], X_tsne[:,1],alpha=0.8,edgecolors="#F50D2F" ,linewidths=0.8, c=y)
axarr[1].set_title("Ham data işlenmeden t-SNE uygulanması" ,color="red")
axarr[1].set_ylabel("2.düzlem", color ="black")
axarr[1].set_xlabel("1.düzlem", color ="black")
axarr[1].set_yticklabels([])
axarr[1].set_xticklabels([])
axarr[1].grid(False)
fig.show()

# Sonuç;
1. Rfm analizi sonucu müşteri gruplarının birbirinden farklı olduğunu ve oluşan kümelere göre aksiyon almamız gerektiğini gördük.
2. oluşturduğumuz sıklık, yenilik ve parasallık kavramları ile bu ayrımı yaparak ilgili dataseti(rfm_data) oluşturduk.
3. Dirsek kuralını kullanarak en uygun küme sayısı seçildi.
4. Datanın önce ve sonrası PCA ve TSNE uygulanarak görselleştirildi. 2. kısımda cluster değişkenine göre renklendirerek anlamlı sonuç arandı, ancak bulunamadı.

# References:
* https://www.kaggle.com/alekhyabotta/customer-segmentation
* https://www.kaggle.com/yugagrawal95/k-means-clustering-using-seaborn-visualization
* https://medium.com/@krglnahmetcan/rfm-analizi-ile-m%C3%BC%C5%9Fteri-segmentasyonu-b3c7233b4243
* https://towardsdatascience.com/feature-extraction-using-principal-component-analysis-a-simplified-visual-demo-e5592ced100a
* https://www.scikit-yb.org/en/latest/api/cluster/elbow.html
* https://towardsdatascience.com/all-about-feature-scaling-bcc0ad75cb35#:~:text=Feature%20scaling%20is%20essential%20for,that%20calculate%20distances%20between%20data.&text=Since%20the%20range%20of%20values,not%20work%20correctly%20without%20normalization.
* AND OF COURSE https://stackoverflow.com/